In [10]:
import pandas as pd

data_dir = '/Users/m221138/RA_plasma_metabolites/data'

drug_profile = 'DAM_comfound_effect_test_v2.csv'
metabolic_profile = 'filled_qc_data/MLR.hd4.ready.ignor.norm.qc.fillna.v2.tsv'

drug_profile_file = '%s/%s' % (data_dir, drug_profile)
metabolic_profile_file = '%s/%s' % (data_dir, metabolic_profile)
selection_profile_file = '%s/%s' % (data_dir,'67_metabolites.csv')


In [11]:
drug_profile_df = pd.read_csv(drug_profile_file)
metabolic_profile_df = pd.read_csv(metabolic_profile_file, sep="\t")
selection_profile_df = pd.read_csv(selection_profile_file)

In [25]:
# selection_profile_df = pd.read_csv(selection_profile_file)
metabolite_list = list(selection_profile_df.iloc[:,0])
print (metabolite_list)
print (len(metabolite_list))

[100015839, 100020414, 100001104, 100015838, 100001253, 100001950, 100001254, 100020546, 415, 250, 171, 100003151, 100000269, 100001092, 565, 100015831, 100001313, 100001391, 100001266, 100001197, 100001257, 100021141, 100000776, 100000257, 100001620, 503, 100004088, 100020204, 100004575, 100000007, 407, 100001577, 100020361, 100001635, 100002466, 100003397, 344, 100020866, 100002254, 100000939, 100000096, 100000042, 100001586, 100001767, 100008933, 796, 1162, 100020427, 100003892, 100002945, 100001051, 1087, 100002488, 100000010, 803, 1090, 572, 100009082, 100020425, 100020423, 1489, 100001510, 1442, 100002911, 100021374, 338, 363]
67


In [18]:
def add_row_values_to_dict (data_df, data_dict, key, index):
    data_dict[key] = list(data_df.iloc[index,1:])
    return data_dict

def add_drug_status_to_dict (data_dict, key, value):
    
    try: data_dict[key].append(value)
    except KeyError: data_dict[key] = [value]
        
    return data_dict

In [26]:
# print (metabolite_list)

[100015839, 100020414, 100001104, 100015838, 100001253, 100001950, 100001254, 100020546, 415, 250, 171, 100003151, 100000269, 100001092, 565, 100015831, 100001313, 100001391, 100001266, 100001197, 100001257, 100021141, 100000776, 100000257, 100001620, 503, 100004088, 100020204, 100004575, 100000007, 407, 100001577, 100020361, 100001635, 100002466, 100003397, 344, 100020866, 100002254, 100000939, 100000096, 100000042, 100001586, 100001767, 100008933, 796, 1162, 100020427, 100003892, 100002945, 100001051, 1087, 100002488, 100000010, 803, 1090, 572, 100009082, 100020425, 100020423, 1489, 100001510, 1442, 100002911, 100021374, 338, 363]


In [20]:
r, c = metabolic_profile_df.shape
data_info_dict = {}
patientID_list = list(metabolic_profile_df.columns.values)[1:]

for i in range(r):
    
    index_name = metabolic_profile_df.iloc[i,0]
    if i == 0: #patientID
        add_row_values_to_dict(metabolic_profile_df, data_info_dict, index_name, i)
    if i == 1: #sex
        add_row_values_to_dict(metabolic_profile_df, data_info_dict, index_name, i)
    if i == 2: #age
        add_row_values_to_dict(metabolic_profile_df, data_info_dict, index_name, i)
    if i > 5:
        index_name = int(index_name)
        if index_name in metabolite_list:
            add_row_values_to_dict(metabolic_profile_df, data_info_dict, index_name, i)

In [21]:
r, c = drug_profile_df.shape

for i in range(r):
    
    mtx_status = drug_profile_df.iloc[i,1]
    
    if mtx_status == "yes":
        mtx_status = 1
    if mtx_status == "no":
        mtx_status = 0
        
    csDMARD_status = drug_profile_df.iloc[i,2]
    tnfi_status = drug_profile_df.iloc[i,3]
    ntnfi_status = drug_profile_df.iloc[i,4]
    pred_status = drug_profile_df.iloc[i,5]
    
    if pred_status == "yes":
        pred_status = 1
    if pred_status == "no":
        pred_status = 0
        
    add_drug_status_to_dict(data_info_dict, "mtx", mtx_status)
    add_drug_status_to_dict(data_info_dict, "csdmard", csDMARD_status)
    add_drug_status_to_dict(data_info_dict, "tnfi", tnfi_status)
    add_drug_status_to_dict(data_info_dict, "ntnfi", ntnfi_status)
    add_drug_status_to_dict(data_info_dict, "pred", pred_status)

In [22]:
# data_info_dict["patientID"] = patientID_list
data_info_dict.keys()
factor_key_list = ['patient_dummyID', 'SEX', 'AGE','mtx', 'csdmard', 'tnfi', 'ntnfi', 'pred']

In [30]:
output_file = '/Users/m221138/RA_plasma_metabolites/data/drug_effect_67_met_profile.txt'
output_txt = open(output_file,'w')

for patientID in patientID_list:
    output_txt.write('\t%s' % patientID)
output_txt.write('\n')

for key in factor_key_list:
    output_txt.write(key)
    for value in data_info_dict[key]:
        output_txt.write('\t%s' % value)
    output_txt.write('\n')  
count = 0
for key in data_info_dict.keys():   
    if key not in factor_key_list:
        count = count + 1
        output_txt.write(str(key))
        for value in data_info_dict[key]:
            output_txt.write('\t%s' % value)
        output_txt.write('\n')  
        